# AI Image Detection

## Submitted by- Vanshika Saini (20115163)

In [4]:
!pip install tensorflow

     -------------------------------------- 272.8/272.8 MB 1.7 MB/s eta 0:00:00
     -------------------------------------- 57.5/57.5 kB 761.8 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 4.2/4.2 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 440.7/440.7 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 4.5 MB/s eta 0:00:00
     -------------------------------------- 65.5/65.5 kB 442.0 kB/s eta 0:00:00
     -------------------------------------- 422.5/422.5 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 2.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to bu

# Importing Libraries

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Add, Input
from tensorflow.keras.models import Model
from keras.layers import Reshape

In [13]:
#reading csv files
df = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

In [14]:
x = df.drop('labels',axis = 1)
y = df['labels']
# x_train = df.drop('labels', axis=1)
# y_train = df['labels']
# x_id = df2['id']
# x_test = df2.drop('id', axis=1)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Model

In [16]:
#1200*1-image ----csv file -20*20*3
input_shape = (1200, 1)

# outputs- 0,1
num_classes = 2

cnn_inputs = Input(shape=input_shape)

#first layer of CNN
x = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_inputs)
x = MaxPooling1D(pool_size=2)(x)
x = Dense(units=2400, activation='relu')(x)
x = Dense(units=1200, activation='relu')(x)
x = Dense(units=600, activation='relu')(x)
x = Dense(units=300, activation='relu')(x)
x = Flatten()(x)
cnn_outputs = Dense(units=300, activation='relu')(x)


def residual_block(inputs, filters, kernel_size):
    x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(inputs)
    x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same')(x)
    x = Add()([x, inputs])
    x = MaxPooling1D(pool_size=2)(x)
    return x

residual_inputs = Reshape((-1, 1))(cnn_outputs)
x = residual_block(residual_inputs, filters=50, kernel_size=4)
x = Conv1D(filters=50, kernel_size=4, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=3)(x)
x = residual_block(x, filters=50, kernel_size=4)
x = residual_block(x, filters=50, kernel_size=4)
x = residual_block(x, filters=50, kernel_size=4)
x = Flatten()(x)
x = Dense(units=100, activation='relu')(x)
x = Dense(units=10, activation='relu')(x)

#using sigmoid(for binary output)
outputs = Dense(units=1, activation='sigmoid')(x)


# Trainig the model

In [17]:
model = Model(inputs=cnn_inputs, outputs=outputs)
#loss calculation optimizer-adam
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#training
model.fit(x_train, y_train, batch_size=32, epochs=25)

Epoch 1/25
132/132 [==============================] - 545s 4s/step - loss: 0.4627 - accuracy: 0.7993
Epoch 2/25
132/132 [==============================] - 528s 4s/step - loss: 0.2932 - accuracy: 0.8676
Epoch 3/25
132/132 [==============================] - 446s 3s/step - loss: 0.2344 - accuracy: 0.8955
Epoch 4/25
132/132 [==============================] - 566s 4s/step - loss: 0.1860 - accuracy: 0.9188
Epoch 5/25
132/132 [==============================] - 473s 4s/step - loss: 0.1346 - accuracy: 0.9462
Epoch 6/25
132/132 [==============================] - 411s 3s/step - loss: 0.1044 - accuracy: 0.9607
Epoch 7/25
132/132 [==============================] - 413s 3s/step - loss: 0.0721 - accuracy: 0.9731
Epoch 8/25
132/132 [==============================] - 408s 3s/step - loss: 0.0568 - accuracy: 0.9786
Epoch 9/25
132/132 [==============================] - 407s 3s/step - loss: 0.0388 - accuracy: 0.9845
Epoch 10/25
132/132 [==============================] - 404s 3s/step - loss: 0.0318 - accura

In [19]:
# predictions = model.predict(x_test) #predictions-array of outputs
# predicted_labels = (predictions > 0.5).astype(int) #converting outputs to binary (using 0.5 scale)
# accuracy = accuracy_score(target, predicted_labels)
# print("Accuracy:", accuracy)

SyntaxError: invalid non-printable character U+00A0 (576734582.py, line 4)

# Testing

In [25]:
from sklearn.metrics import accuracy_score

predictions = model.predict(x_test)  # predictions-array of outputs
predicted_labels = (predictions > 0.5).astype(int)  # converting outputs to binary (using 0.5 scale)
accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy)



33/33 [==============================] - 38s 1s/step
Accuracy: 0.8580952380952381


In [ ]:
#for complete dataset
result_df = pd.DataFrame({'id': x_id, 'labels': predicted_labels.flatten()})
result_df.to_csv('predictions.csv', index=False)
